# Computational Experiments on Maass Forms

In [1]:
T = matrix([[1, 1], [0, 1]])
S = matrix([[0, -1], [1, 0]])
Id = matrix([[1, 0], [0, 1]])

def act(gamma, z):
    """
    Computes the action of matrix gamma on point z
    """
    a, b = gamma[0]
    c, d = gamma[1]
    return (a*z + b)/ (c*z + d)

def is_in_fund_domain(z):
    """
    True if z is in fundamental domain, False otherwise.
    """
    x = z.real()
    if x < -1/2 or x > 1/2:
        return False
    if z.norm() < 1:
        return False
    return True


def pullback(z):
    """
    Returns matrix gamma, point z^*, where gamma z = z^* and z^* is in the fundamental domain
    """
    gamma = Id
    while not is_in_fund_domain(z):
        x, y = z.real(), z.imag()
        xshift = -floor(x + .5)
        gamma = T**xshift * gamma
        z = x + xshift + i*y
        if z.norm() < 1:
            z = act(S, z)
            gamma = S * gamma
    return gamma, z

In [2]:
pullback(CC(3.75 + .25*I))

(
[-2  7]                    
[ 1 -4], 2.00000000000000*I
)

In [3]:
pullback(CDF(3.75 + .25*I))

(
[-2  7]                                               
[ 1 -4], -4.440892098500626e-16 + 1.9999999999999996*I
)

In [4]:
bessel_K(CC(1.0), CC(3.0))

0.0401564311281942

In [5]:
bessel_K(CDF(1.0), CDF(3.0))

0.040156431128194184

In [6]:
def kappa(r, u):
    return (exp(CC(pi) * CC(r) / 2) * bessel_K(CC(i*r), CC(u)))
#def kappa(r, u):
#    return bessel_K(CDF(i*r), CDF(u))

In [7]:
def make_zj_list(Y, Q=50):
    """
    Returns list of z_j = x_j + iY
    """
    ret = []
    for j in range(1-Q, Q+1):
        ret.append((1/(2*Q)*(j - 0.5) + i*Y))
    return ret

def make_zjstar_list(zjlist):
    """
    Returns list of z_j^* in same order as zjlist
    """
    return list(map(lambda zj: pullback(zj)[1], zjlist))

In [8]:
test_zj_list = make_zj_list(0.5, Q=4)
for elem in test_zj_list:
    print(elem)

-0.437500000000000 + 0.500000000000000*I
-0.312500000000000 + 0.500000000000000*I
-0.187500000000000 + 0.500000000000000*I
-0.0625000000000000 + 0.500000000000000*I
0.0625000000000000 + 0.500000000000000*I
0.187500000000000 + 0.500000000000000*I
0.312500000000000 + 0.500000000000000*I
0.437500000000000 + 0.500000000000000*I


In [9]:
test_zjstar_list = make_zjstar_list(test_zj_list)
for elem in test_zjstar_list:
    print(elem)

-0.00884955752212391 + 1.13274336283186*I
-0.101123595505618 + 1.43820224719101*I
-0.342465753424658 + 1.75342465753425*I
0.246153846153846 + 1.96923076923077*I
-0.246153846153846 + 1.96923076923077*I
0.342465753424658 + 1.75342465753425*I
0.101123595505618 + 1.43820224719101*I
0.00884955752212391 + 1.13274336283186*I


In [10]:
def perform_sanity_check_zjstar(zjlist, zjstarlist):
    for zj, zjstar in zip(zjlist, zjstarlist):
        mat, _ = pullback(zj)
        assert (act(mat, CC(zj)) - zjstar).norm() < 0.00000001
    print("Passed.")
perform_sanity_check_zjstar(test_zj_list, test_zjstar_list)

Passed.


In [11]:
def V(n, l, zjlist, r, Q, zjstarlist=None):
    """
    Return the constant V(n, l) --- which also depends on zj, r, and Q --- as
    defined above.
    """
    if zjstarlist is None:
        zjstarlist = make_zjstar_list(zjlist)
    ret = CDF(0)
    assert len(zjstarlist) == 2*Q
    for zj, zjs in zip(zjlist, zjstarlist):
        xj = zj.real()
        xjs = zjs.real()
        yjs = zjs.imag()
        ret += (sqrt(yjs)) * kappa(r, 2 * pi * abs(l) * yjs) * (exp(2*pi*i*(l * xjs - n * xj)))
    ret = ret / (2*Q)
    return CDF(ret)

## Large Speedup TODO

Essentially all computation time is spent currently computing $V(n, l)$ in order to make the matrix. There are $Q^2$ matrix entries, each formed from a sum of $Q$ values. And right now, I'm computing all $Q^3$ K-Bessel functions. But this is excessive. In fact, there are at most $Q^2$ K-Bessel functions that need to be computed:
$$ K_{ir}(2 \pi \lvert \ell \rvert y_j^*),$$
where $1 \leq \lvert \ell \rvert \leq Q$ and $j \in [-Q, Q]$.

Rewriting the matrix-generation code to take advantage of this with caching would make it a $Q^2$ operation (assuming that the Bessel functions are where all of the computation time is being spent).

Section 3.4 of [this paper](http://www2.math.uu.se/~astrombe/papers/kbessel.pdf) gives a further improvement for repeated honing in, as only the values of $r$ are changing. Sections 3.2 and 3.3 might give improvements for an individual run --- I don't quite understand the implied algorithms.

The matrix-generation code should also be vectorized as much as possible.

In [12]:
V(3, 1, test_zj_list, 9.5**.5, 4)

-0.0016458022812606853 + 1.6263032587282567e-19*I

In [13]:
import time

In [14]:
now = time.time()
print(V(3, 1, test_zj_list, 9.5**.5, 4))
print("Took {}".format(time.time() - now))

-0.0016458022812606853 + 1.6263032587282567e-19*I
Took 0.047792673110961914


In [15]:
def make_matrix(size, Y, r, withtime=True):
    if withtime:
        now = time.time()
    Q = size
    zjlist = make_zj_list(Y, Q)
    zjstarlist = make_zjstar_list(zjlist)
    if withtime:
        print("lists made in {}".format(time.time() - now))
    matrix = []
    B = []
    for n in range(-Q, Q+1):
        if withtime:
            now = time.time()
        if n == 0:
            continue
        row = []
        for l in range(-Q, Q+1):
            if l == 0:
                continue
            entry = CDF(V(n, l, zjlist, CDF(r), CDF(Q), zjstarlist=zjstarlist))
            if n == l:
                #entry = entry - sqrt(Y) * bessel_K(i*r, 2 * pi * abs(n) * Y)
                entry = entry - CDF(sqrt(Y)) * kappa(r, 2 * pi * abs(n) * Y)
            if l != 1:
                row.append(entry.n())
            else:
                B.append([-entry.n()])
                #if n == 1:
                #    print(V(n, l, zjlist, r, Q, zjstarlist=zjstarlist))
                #    print(sqrt(Y) * kappa(r, 2 * pi * abs(n) * Y))
        matrix.append(row)
        if withtime:
            print("row {} computed in {} seconds".format(n, time.time() - now))
    return matrix, B

In [16]:
Q = 8
Y = 0.28
r = 9.53369526135
now = time.time()
#testmat, testB = make_matrix(20, 0.5, 3)
testmat, testB = make_matrix(Q, Y, r)
print("Done in {}".format(time.time() - now))

lists made in 0.03257441520690918
row -8 computed in 0.6628572940826416 seconds
row -7 computed in 0.6433212757110596 seconds
row -6 computed in 0.6426317691802979 seconds
row -5 computed in 0.6682097911834717 seconds
row -4 computed in 0.6700131893157959 seconds
row -3 computed in 0.6402966976165771 seconds
row -2 computed in 0.663346529006958 seconds
row -1 computed in 0.6863448619842529 seconds
row 1 computed in 0.7095780372619629 seconds
row 2 computed in 0.9151451587677002 seconds
row 3 computed in 0.798314094543457 seconds
row 4 computed in 0.9088175296783447 seconds
row 5 computed in 0.8387832641601562 seconds
row 6 computed in 0.8383357524871826 seconds
row 7 computed in 0.8320372104644775 seconds
row 8 computed in 0.8334674835205078 seconds
Done in 11.985921859741211


## Contents of tiny cython file

```python
import sage.libs.mpmath.all as mpmath

mpmath_ctx = mpmath.fp


def cykappa(r, u):
    return mpmath_ctx.besselk(1j*r, u)

def cykappa_with_mult(r, u):
    pi = mpmath_ctx.pi
    cdef double exp_Pih_R=mpmath_ctx.exp(pi * r / 2.)
    return mpmath_ctx.besselk(1j*r, u) * exp_Pih_R

def cyV(n, l, r, Q, zjlist, zjstarlist):
    pi = mpmath_ctx.pi
    cdef double exp_Pih_R=mpmath_ctx.exp(pi * r / 2.)
    cdef double complex ret = 0
    for zj, zjs in zip(zjlist, zjstarlist):
        xj = zj.real()
        xjs = zjs.real()
        yjs = zjs.imag()
        ret += mpmath_ctx.sqrt(yjs) * exp_Pih_R * cykappa(r, 2*pi*abs(l)*yjs) \
               * mpmath_ctx.exp(2*pi*1j*(l * xjs - n * xj))
    ret = ret / (2*Q)
    return ret
```

In [17]:
load("maass-cython.spyx")

Compiling ./maass-cython.spyx...


In [18]:
cykappa_with_mult(1, 3+i)

(0.06333777981976761-0.12791312356062806j)

In [19]:
kappa(1, 3+i)

0.0633377798197707 - 0.127913123560625*I

In [20]:
test_zj_starlist = make_zjstar_list(test_zj_list)

In [21]:
cyV(3, 1, 9.5**.5, 4, test_zj_list, test_zj_starlist)

(-0.0016458022789573614+0j)

In [22]:
V(3, 1, test_zj_list, 9.5**.5, 4)

-0.0016458022812606853 + 1.6263032587282567e-19*I

In [23]:
%time V(3, 1, test_zj_list, 9.5**.5, 4)

CPU times: user 47 ms, sys: 3.99 ms, total: 51 ms
Wall time: 49.6 ms


-0.0016458022812606853 + 1.6263032587282567e-19*I

In [24]:
%time cyV(3, 1, 9.5**.5, 4, test_zj_list, test_zj_starlist)

CPU times: user 3.8 ms, sys: 0 ns, total: 3.8 ms
Wall time: 3.81 ms


(-0.0016458022789573614+0j)

In [25]:
def make_matrix_with_some_cy(size, Y, r, withtime=True):
    if withtime:
        now = time.time()
    Q = size
    zjlist = make_zj_list(Y, Q)
    zjstarlist = make_zjstar_list(zjlist)
    if withtime:
        print("lists made in {}".format(time.time() - now))
    matrix = []
    B = []
    for n in range(-Q, Q+1):
        if withtime:
            now = time.time()
        if n == 0:
            continue
        row = []
        for l in range(-Q, Q+1):
            if l == 0:
                continue
            entry = cyV(n, l, CDF(r), CDF(Q), zjlist, zjstarlist=zjstarlist)
            if n == l:
                #entry = entry - sqrt(Y) * bessel_K(i*r, 2 * pi * abs(n) * Y)
                entry = entry - CDF(sqrt(Y)) * cykappa_with_mult(r, 2 * pi * abs(n) * Y)
            if l != 1:
                row.append(entry)
            else:
                B.append([-entry])
                #if n == 1:
                #    print(V(n, l, zjlist, r, Q, zjstarlist=zjstarlist))
                #    print(sqrt(Y) * kappa(r, 2 * pi * abs(n) * Y))
        matrix.append(row)
        if withtime:
            print("row {} computed in {} seconds".format(n, time.time() - now))
    return matrix, B

In [26]:
Q = 8
Y = 0.28
r = 9.53369526135
#testmat, testB = make_matrix(20, 0.5, 3)
now = time.time()
testmat, testB = make_matrix_with_some_cy(Q, Y, r)
print("Done in {}".format(time.time() - now))

lists made in 0.02890300750732422
row -8 computed in 0.06800627708435059 seconds
row -7 computed in 0.06681990623474121 seconds
row -6 computed in 0.06819319725036621 seconds
row -5 computed in 0.06733584403991699 seconds
row -4 computed in 0.06668877601623535 seconds
row -3 computed in 0.06763362884521484 seconds
row -2 computed in 0.06885933876037598 seconds
row -1 computed in 0.06583356857299805 seconds
row 1 computed in 0.06247305870056152 seconds
row 2 computed in 0.06531190872192383 seconds
row 3 computed in 0.06538820266723633 seconds
row 4 computed in 0.06538653373718262 seconds
row 5 computed in 0.06470918655395508 seconds
row 6 computed in 0.06978678703308105 seconds
row 7 computed in 0.06601834297180176 seconds
row 8 computed in 0.06732916831970215 seconds
Done in 1.0968985557556152


In [27]:
def solve_for_coeffs(Q, Y, r):
    mat, B = make_matrix_with_some_cy(Q, Y, r, withtime=False)
    preinversiontime = time.time()
    mm = matrix(CC, mat[1:])
    Bm = matrix(CC, B[1:])
    res = mm.inverse() * Bm
    print("Inversion done in {}".format(time.time() - preinversiontime))
    indices = [i for i in range(-Q, Q) if i != 0 and i != 1]
    resdict = dict()
    for index, re in zip(indices, res):
        resdict[index] = re[0].real()
    return resdict

In [28]:
%time coeffdict = solve_for_coeffs(Q, Y, r)

Inversion done in 0.013143301010131836
CPU times: user 1.07 s, sys: 3.91 ms, total: 1.07 s
Wall time: 1.07 s


In [29]:
coeffdict

{-8: 1.87322686737660e9,
 -7: 0.793986924762749,
 -6: -0.488696770726782,
 -5: 0.290607870282857,
 -4: -0.141331593018057,
 -3: 0.456186618172060,
 -2: 1.06833016591228,
 -1: -0.999999234922817,
 2: -1.06833082621984,
 3: -0.456187120225734,
 4: 0.141331341278230,
 5: -0.290607775330800,
 6: 0.488696650679484,
 7: -0.793986654374832}

In [30]:
def error_func(r, Y1=0.28, Y2=0.3, Q=8, power=2):
    coeffdict1 = solve_for_coeffs(Q, Y1, r)
    coeffdict2 = solve_for_coeffs(Q, Y2, r)
    return ((coeffdict1[2] - coeffdict2[2])**power
            + (coeffdict1[3] - coeffdict2[3]) **power
            + (coeffdict1[5] - coeffdict2[5])**power)

In [31]:
%time error_func(r)

Inversion done in 0.005559206008911133
Inversion done in 0.005434274673461914
CPU times: user 2.21 s, sys: 38 µs, total: 2.21 s
Wall time: 2.2 s


1.75925992991161e-9

In [32]:
def secant_method_single_iteration(r1, r2, Q=8, power=2):
    e1 = error_func(r1, Q=Q, power=power)
    e2 = error_func(r2, Q=Q, power=power)
    return r1 - e1*(r1 - r2)/(e1 - e2)

In [33]:
r1 = 9.5
r2 = 9.55
rlist = [r1, r2]
NOW = time.time()
now = time.time()
for idx in range(10):
    rnew = secant_method_single_iteration(rlist[-2], rlist[-1], power=2)
    rlist.append(rnew)
    print("computed r: {}      time: {}".format(rnew, time.time() - now))
    now = time.time()
print("Done in {}".format(time.time() - NOW))

Inversion done in 0.00494074821472168
Inversion done in 0.0050106048583984375
Inversion done in 0.004925251007080078
Inversion done in 0.004859447479248047
computed r: 9.49727787715483      time: 4.3293445110321045
Inversion done in 0.004935264587402344
Inversion done in 0.0050411224365234375
Inversion done in 0.004921913146972656
Inversion done in 0.005501747131347656
computed r: 9.49419505507187      time: 4.285295724868774
Inversion done in 0.005172014236450195
Inversion done in 0.0049092769622802734
Inversion done in 0.004960298538208008
Inversion done in 0.004971981048583984
computed r: 9.54086107605566      time: 4.35932469367981
Inversion done in 0.004830598831176758
Inversion done in 0.0049779415130615234
Inversion done in 0.004949808120727539
Inversion done in 0.005103349685668945
computed r: 9.58660052314911      time: 4.305162191390991
Inversion done in 0.005700588226318359
Inversion done in 0.004929304122924805
Inversion done in 0.0049135684967041016
Inversion done in 0.004

In [34]:
r1 = 9.5
r2 = 9.55
rlist = [r1, r2]
NOW = time.time()
now = time.time()
for idx in range(10):
    rnew = secant_method_single_iteration(rlist[-2], rlist[-1], power=1)
    rlist.append(rnew)
    print("computed r: {}      time: {}".format(rnew, time.time() - now))
    now = time.time()
print("Done in {}".format(time.time() - NOW))

Inversion done in 0.0048999786376953125
Inversion done in 0.005072116851806641
Inversion done in 0.0051136016845703125
Inversion done in 0.0050792694091796875
computed r: 9.50928635143440      time: 4.302231550216675
Inversion done in 0.0050313472747802734
Inversion done in 0.005052089691162109
Inversion done in 0.004992485046386719
Inversion done in 0.0050067901611328125
computed r: 9.51594812823334      time: 4.314939022064209
Inversion done in 0.0050127506256103516
Inversion done in 0.004965782165527344
Inversion done in 0.005096435546875
Inversion done in 0.005613088607788086
computed r: 9.55109466860222      time: 4.276482105255127
Inversion done in 0.005095720291137695
Inversion done in 0.005231142044067383
Inversion done in 0.005057573318481445
Inversion done in 0.004981517791748047
computed r: 9.51984614699136      time: 4.285640478134155
Inversion done in 0.005049228668212891
Inversion done in 0.005057573318481445
Inversion done in 0.004965066909790039
Inversion done in 0.0050

In [35]:
r1 = 9.5
r2 = 9.55
rlist = [r1, r2]
NOW = time.time()
now = time.time()
for idx in range(10):
    rnew = secant_method_single_iteration(rlist[-2], rlist[-1], Q=6)
    rlist.append(rnew)
    print("computed r: {}      time: {}".format(rnew, time.time() - now))
    now = time.time()
print("Done in {}".format(time.time() - NOW))

Inversion done in 0.0035152435302734375
Inversion done in 0.0027489662170410156
Inversion done in 0.002672910690307617
Inversion done in 0.0027647018432617188
computed r: 9.59387047826304      time: 2.0381362438201904
Inversion done in 0.002574443817138672
Inversion done in 0.0026612281799316406
Inversion done in 0.002622365951538086
Inversion done in 0.002717256546020508
computed r: 9.54812422154999      time: 2.027291774749756
Inversion done in 0.0026547908782958984
Inversion done in 0.002655506134033203
Inversion done in 0.0033588409423828125
Inversion done in 0.06827116012573242
computed r: 9.54664217265963      time: 2.1221470832824707
Inversion done in 0.002504110336303711
Inversion done in 0.002727031707763672
Inversion done in 0.0027239322662353516
Inversion done in 0.002712249755859375
computed r: 9.54102948022800      time: 2.025088310241699
Inversion done in 0.002660512924194336
Inversion done in 0.002702474594116211
Inversion done in 0.0027251243591308594
Inversion done in 

In [36]:
error_func(r, Q=6)

Inversion done in 0.00270843505859375
Inversion done in 0.0026862621307373047


0.00120185116773810

I don't quite understand the required bounds on $Q$ at the moment. But surprisingly small values of $Q$ work surprisingly well.

In [37]:
r1 = 12.0
r2 = 12.5
rlist = [r1, r2]
NOW = time.time()
now = time.time()
for idx in range(20):
    rnew = secant_method_single_iteration(rlist[-2], rlist[-1], Q=8)
    rlist.append(rnew)
    print("computed r: {}      time: {}".format(rnew, time.time() - now))
    now = time.time()
print("Done in {}".format(time.time() - NOW))

Inversion done in 0.0051457881927490234
Inversion done in 0.006009578704833984
Inversion done in 0.005298614501953125
Inversion done in 0.005230903625488281
computed r: 11.9729254449747      time: 4.7046849727630615
Inversion done in 0.005159854888916016
Inversion done in 0.005223751068115234
Inversion done in 0.005034923553466797
Inversion done in 0.005239009857177734
computed r: 11.9416688434772      time: 4.723443269729614
Inversion done in 0.005163669586181641
Inversion done in 0.005152702331542969
Inversion done in 0.005980491638183594
Inversion done in 0.005321502685546875
computed r: 12.1450949287850      time: 4.645413160324097
Inversion done in 0.0052263736724853516
Inversion done in 0.005236387252807617
Inversion done in 0.005252361297607422
Inversion done in 0.005209684371948242
computed r: 12.2230004572527      time: 4.645998954772949
Inversion done in 0.005182743072509766
Inversion done in 0.005239963531494141
Inversion done in 0.0051343441009521484
Inversion done in 0.006

In [38]:
r1 = 12.15
r2 = 12.2
rlist = [r1, r2]
NOW = time.time()
now = time.time()
for idx in range(10):
    rnew = secant_method_single_iteration(rlist[-2], rlist[-1], Q=6)
    rlist.append(rnew)
    print("computed r: {}      time: {}".format(rnew, time.time() - now))
    now = time.time()
print("Done in {}".format(time.time() - NOW))

Inversion done in 0.002987384796142578
Inversion done in 0.003139019012451172
Inversion done in 0.002670764923095703
Inversion done in 0.003651857376098633
computed r: 12.2686437411483      time: 2.285745143890381
Inversion done in 0.002882719039916992
Inversion done in 0.003132343292236328
Inversion done in 0.0028274059295654297
Inversion done in 0.0029087066650390625
computed r: 12.1999812501054      time: 2.3465421199798584
Inversion done in 0.002807140350341797
Inversion done in 0.0027446746826171875
Inversion done in 0.0028433799743652344
Inversion done in 0.002790212631225586
computed r: 12.1999625422140      time: 2.2490718364715576
Inversion done in 0.0027322769165039062
Inversion done in 0.0028107166290283203
Inversion done in 0.002768278121948242
Inversion done in 0.003839254379272461
computed r: 12.1925157918053      time: 2.279813051223755
Inversion done in 0.0028047561645507812
Inversion done in 0.0028917789459228516
Inversion done in 0.0028030872344970703
Inversion done i

I am rapidly running into precision errors. Many of these are caused, I think, by the larger-than-necessary K-bessel functions. I'm using $K_{ir}(u)$ instead of Andy's $\kappa_{ir}(u)$, simply because the libraries I have can compute these quickly. I should try to find a better K-bessel computation.

In [39]:
r1 = 12.15
r2 = 12.2
rlist = [r1, r2]
NOW = time.time()
now = time.time()
for idx in range(10):
    rnew = secant_method_single_iteration(rlist[-2], rlist[-1], Q=10)
    rlist.append(rnew)
    print("computed r: {}      time: {}".format(rnew, time.time() - now))
    now = time.time()
print("Done in {}".format(time.time() - NOW))

Inversion done in 0.010133981704711914
Inversion done in 0.009264945983886719
Inversion done in 0.009323835372924805
Inversion done in 0.011097908020019531
computed r: 12.1495446393491      time: 8.355088472366333
Inversion done in 0.00950479507446289
Inversion done in 0.010741949081420898
Inversion done in 0.009484529495239258
Inversion done in 0.00935220718383789
computed r: 12.1490742084784      time: 8.347170114517212
Inversion done in 0.009395122528076172
Inversion done in 0.009287595748901367
Inversion done in 0.009331941604614258
Inversion done in 0.011080503463745117
computed r: 12.1694270709510      time: 8.331717491149902
Inversion done in 0.009650468826293945
Inversion done in 0.009494304656982422
Inversion done in 0.009322166442871094
Inversion done in 0.009551525115966797
computed r: 12.1705659328399      time: 8.360275268554688
Inversion done in 0.010320663452148438
Inversion done in 0.011347293853759766
Inversion done in 0.010271310806274414
Inversion done in 0.009859323

There are apparently precision problems; see the following computation. $Q = 20$ is much higher, but the actual convergence is not any faster.

In [40]:
r1 = 12.15
r2 = 12.2
rlist = [r1, r2]
NOW = time.time()
now = time.time()
for idx in range(10):
    rnew = secant_method_single_iteration(rlist[-2], rlist[-1], Q=20)
    rlist.append(rnew)
    print("computed r: {}      time: {}".format(rnew, time.time() - now))
    now = time.time()
print("Done in {}".format(time.time() - NOW))

Inversion done in 0.06637048721313477
Inversion done in 0.12478208541870117
Inversion done in 0.06429123878479004
Inversion done in 0.06825780868530273
computed r: 12.1013321704435      time: 55.27284812927246
Inversion done in 0.08541512489318848
Inversion done in 0.06437945365905762
Inversion done in 0.07300662994384766
Inversion done in 0.06478047370910645
computed r: 12.2206422540283      time: 54.621328830718994
Inversion done in 0.06634902954101562
Inversion done in 0.06354093551635742
Inversion done in 0.07012414932250977
Inversion done in 0.06394267082214355
computed r: 10.8987101117853      time: 54.14669919013977
Inversion done in 0.06333732604980469
Inversion done in 0.06509089469909668
Inversion done in 0.06349635124206543
Inversion done in 0.06322932243347168
computed r: 12.2228958502830      time: 53.6555712223053
Inversion done in 0.06446576118469238
Inversion done in 0.061765432357788086
Inversion done in 0.06392073631286621
Inversion done in 0.06449246406555176
compute

Ah, or maybe everything is improved if we use the power 1 in the error function, instead of the power 2. I had thought finding a minimum would be superior to finding a zero, but not with the secant method setup I suppose.

In [41]:
r1 = 12.15
r2 = 12.2
rlist = [r1, r2]
NOW = time.time()
now = time.time()
for idx in range(10):
    rnew = secant_method_single_iteration(rlist[-2], rlist[-1], Q=20, power=1)
    rlist.append(rnew)
    print("computed r: {}      time: {}".format(rnew, time.time() - now))
    now = time.time()
print("Done in {}".format(time.time() - NOW))

Inversion done in 0.06879520416259766
Inversion done in 0.06493043899536133
Inversion done in 0.06588220596313477
Inversion done in 0.05297350883483887
computed r: 12.1741257896385      time: 53.42044758796692
Inversion done in 0.05907082557678223
Inversion done in 0.053086280822753906
Inversion done in 0.0600428581237793
Inversion done in 0.05610370635986328
computed r: 12.1729719029430      time: 46.20607328414917
Inversion done in 0.051614999771118164
Inversion done in 0.05511617660522461
Inversion done in 0.0531153678894043
Inversion done in 0.05326652526855469
computed r: 12.1730084069161      time: 45.5474808216095
Inversion done in 0.05291485786437988
Inversion done in 0.13964033126831055
Inversion done in 0.06515169143676758
Inversion done in 0.052924394607543945
computed r: 12.1730083260559      time: 49.9977605342865
Inversion done in 0.05802154541015625
Inversion done in 0.0569610595703125
Inversion done in 0.05191159248352051
Inversion done in 0.05376386642456055
computed r

In [42]:
def make_single_matrix(size, Y, r, withtime=True):
    if withtime:
        now = time.time()
    Q = size
    zjlist = make_zj_list(Y, Q)
    zjstarlist = make_zjstar_list(zjlist)
    if withtime:
        print("lists made in {}".format(time.time() - now))
    matrix = []
    B = []
    for n in range(-Q, Q+1):
        if withtime:
            now = time.time()
        if n == 0:
            continue
        row = []
        for l in range(-Q, Q+1):
            if l == 0:
                continue
            entry = CDF(V(n, l, zjlist, CDF(r), CDF(Q), zjstarlist=zjstarlist))
            if n == l:
                #entry = entry - sqrt(Y) * bessel_K(i*r, 2 * pi * abs(n) * Y)
                entry = entry - CDF(sqrt(Y)) * kappa(r, 2 * pi * abs(n) * Y)
            if l != 1:
                row.append(entry.n())
            else:
                B.append([-entry.n()])
                #if n == 1:
                #    print(V(n, l, zjlist, r, Q, zjstarlist=zjstarlist))
                #    print(sqrt(Y) * kappa(r, 2 * pi * abs(n) * Y))
        matrix.append(row)
        if withtime:
            print("row {} computed in {} seconds".format(n, time.time() - now))
    return matrix, B

In [ ]:
make_single_matrixe_matrixl